**INSTALLING AND IMPORTING LIBRARIES**

In [ ]:
!pip install emoji
!pip install pyspellchecker

In [ ]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

import re

import emoji

from spellchecker import SpellChecker
spell = SpellChecker()

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import random
# from sklearn import metrics, model_selection, preprocessing

from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
nltk.download("punkt")
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

**LOADING THE DATASET**

In [ ]:
df = pd.read_csv("bonus_cov19_sa.csv")
df.head()

In [ ]:
df = df.drop(["ID", "Unnamed: 0"], axis = 1)

**EDA TO BE DONE**

**DATA PRE-PROCESSING**

In [ ]:
# Define a function to correct spelling mistakes in a string
def correct_spelling(text):
    words = text.split()
    corrected_words = []
    for word in words:
        corrected_word = spell.correction(word)
        if corrected_word != None:
          corrected_words.append(corrected_word)
        # print(corrected_word)
    corrected_text = ' '.join(corrected_words)
    return corrected_text

In [ ]:
abbreviations = {
    "w/": "with",
    "w/o": "without",
    "msg": "message",
    "u": "you",
    "r": "are",
    "lol": "laughing out loud",
    "np": "no problem",
    "LOL": "laughing out loud",
    "XD" : "laugh",
    "xd" : "laugh"
}

# Define a function to correct abbreviations in a string
def correct_abbreviations(text, abbreviations):
    words = text.split()
    corrected_words = []
    for word in words:
        if word in abbreviations:
            corrected_word = abbreviations[word]
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)
    corrected_text = ' '.join(corrected_words)
    return corrected_text

In [ ]:
def preprocess_tweets(tweet):
  # Convert all words to lower case
  tweet = tweet.lower()

  # Remove all punctuations
  tweet = tweet.translate(str.maketrans('', '', string.punctuation))

  # Remove all stop words
  # Tokenize the sentence
  tokens = nltk.word_tokenize(tweet)
  # Remove stop words
  clean_tokens = [token for token in tokens if not token.lower() in stop_words]
  # Join the tokens back into a sentence
  tweet = ' '.join(clean_tokens)

  # Removing HTML Tags, URLs and Website Links
  # Removing Website Links
  temp_text = tweet.split()

  for i in temp_text:
    if i[-4:] == ".com" or i[:4] == "www.":
      temp_text.remove(i)

  tweet = ' '.join(temp_text)

  # Remove HTML tags
  clean_text = re.sub('<[^<]+?>', '', tweet)

  # Remove URLs
  clean_text = re.sub(r'http\S+', '', clean_text)

  tweet = clean_text

  # Remove numbers
  tweet = re.sub(r'\d+', '', tweet)

  # Deemojize Emojis
  tweet = emoji.demojize(tweet)

  # Correct Abbreviations
  tweet = correct_abbreviations(tweet, abbreviations)

  # Remove mentions
  tweet = re.sub(r'@\w+', '', tweet)

  # Stemming
  # Tokenize the sentence
  tokens = nltk.word_tokenize(tweet)

  # Stem the tokens
  stemmed_tokens = [stemmer.stem(token) for token in tokens]

  # Join the stemmed tokens back into a sentence
  tweet = ' '.join(stemmed_tokens)

    # Spellcheck
  try:
    tweet = correct_spelling(tweet)
  except Exception as e:
    print(e)

  return tweet  

In [ ]:
df["Tweet"] = df["Tweet"].map(preprocess_tweets)
df.head()

In [ ]:
# Downloading the .csv of the preprocessed Dataframe

df.to_csv("bonus_cov19_sa_preprocessed.csv",index = False)